In [248]:
import numpy as np
import pandas as pd

In [249]:
def rand_design_matrix(experiments, features) -> pd.DataFrame:
    """
    :param int experiments: Number of Experiments to design
    :param int features: Number of features to include in the design


    Generate a random starting design matrix
    """
    df = pd.DataFrame(np.random.random((experiments, features)))
    df.columns = ['x' + str(x) for x in list(range(features))]
    return df

In [250]:
def get_model_matrix(data, order, interactions_only=False, bias=True) -> pd.DataFrame:
    """
    :param pd.DataFrame data: Design matrix
    :param int order: Order of the polynomial (1-main effects, 2-quadratic effects, ...)
    :param bool interactions_only: Include terms as x1^2 or not
    :param bool bias: Include a beta_0 on the design matrix or not


    Generate the model matrix of a design matrix (argument)
    """
    from sklearn.preprocessing import PolynomialFeatures

    poly = PolynomialFeatures(order, interaction_only=interactions_only, include_bias=bias)
    df = pd.DataFrame(poly.fit_transform(data))
    df.columns=poly.get_feature_names(data.columns)
    return df

In [251]:
all_dets = []
designs, histories = pd.DataFrame(), pd.DataFrame()

levels = [-1., 0, 1.]

n_exp = 25
n_feat = 5
epochs = 1000


for epoch in range(epochs):
    design_matrix = rand_design_matrix(experiments=n_exp, features=n_feat)
    for row in range(n_exp):
        for col in range(n_feat):
            coordinate_det = []
            for count, level in enumerate(levels):

                design_matrix.iat[row, col] = levels[count] # change the design matrix to each level element
                model_matrix = get_model_matrix(data=design_matrix, order=2, interactions_only=False, bias=True) # calculate the design matrix
                det = np.linalg.det(model_matrix.T @ model_matrix) # calculate the determinant
                coordinate_det.append(det) # write result on a list to find the max (and most importantly the max position)

            all_dets.append([row, col, *coordinate_det]) # keep entire history (unpack the coordinate_det to make all_dets a pd.DataFrame)
            design_matrix.iat[row, col] = levels[coordinate_det.index(max(coordinate_det))] # change the design matrix[row, col] element to the element of leves at the max position

    design_matrix['epoch'] = epoch
    history = pd.DataFrame(all_dets)
    history['max'] = history.iloc[:,2:].max(axis=1)
    history['epoch'] = epoch

    designs = designs.append(design_matrix, ignore_index=True)
    histories = histories.append(history, ignore_index=True)


KeyboardInterrupt: 

In [ ]:
histories.groupby(by='epoch').max('max')['max']

In [ ]:
x1 = rand_design_matrix(experiments=12, features=3)
x2 = rand_design_matrix(experiments=9, features=3)

In [ ]:
dfs = []
x1['origin'] = 'x1'
x2['origin'] = 'x2'
dfs.extend([x1, x2])
df = pd.concat(dfs)
df